In [1]:
import pandas as pd
import numpy as np
import folium
import json
import requests

In [2]:
#get data from firebase website using filtering
r = requests.get('https://fight-on-fire-off-4f39b-default-rtdb.firebaseio.com/.json') 

#convert to json object
r = r.json()

#sort data using key - customer id
df = json.dumps(r)


#covert data back to pandas dataframe for easier display
data = json.loads(df)
#df = pd.DataFrame(data)
#df.reset_index(level=0, inplace=True) #???

df = pd.read_json(df, orient ='records')
#print(df)

In [3]:
loc = df[ ['LAT', 'LON'] ].to_numpy()

In [5]:
from folium.plugins import Search
from folium.plugins import MarkerCluster

map_m = folium.Map( location= ( df.LAT.mean(), df.LON.mean() ), title = "Facilities", zoom_start= 6)
marker_cluster = MarkerCluster().add_to(map_m)

for i, v in df.iterrows():
    popup = """
    COUNTY: <b>%s</b><br>
    ADDRESS : <b>%s</b><br>
    CITY : <b>%s</b><br>
    ZIP : <b>%s</b><br>
    PHONE_NUM: <b>%s</b><br>
    """ % (v['COUNTY'], v['ADDRESS'], 
           v['CITY'], v['ZIP'], 
           v['PHONE_NUM'])

    folium.Circle(
        location = loc[i].tolist(),
        tooltip = popup,
        radius = 1e3, 
        color = 'green', 
        fill = True 
    ).add_to(map_m)
    
    
    folium.Marker(
        location= loc[i].tolist(),
        popup = v["CITY"],
        name = v["CITY"],
        icon= None
    ).add_to(marker_cluster)
    
#add search bar
servicesearch = Search(
    layer=marker_cluster,
    search_label= "name",
    placeholder='Search City',
    collapsed=False,
).add_to(map_m)


map_m 

In [6]:
#save folium map for app deployment
map_m.save(r'./facilities_map.html')